In [ ]:
# import libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time

def safe_find_element_text(element, by, value, default=""):
    try:
        return element.find_element(by, value).text
    except Exception:
        return default

def safe_find_element_attribute(element, by, value, attribute, default=""):
    try:
        return element.find_element(by, value).get_attribute(attribute)
    except Exception:
        return default

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

all_pages = []

for page_num in range(45,161):
    main_url = "https://hewlett.org/grants/?keyword=Africa&sort=date&current_page=" + str(page_num)
    # load the web page
    driver.get(main_url)

    # set maximum time to load the web page in seconds
    driver.implicitly_wait(5)

    grants = driver.find_elements(By.CLASS_NAME, "grant-listing-callout")

    for grant_link in grants:
    
        link = grant_link.find_element(By.TAG_NAME, "a")
        link.click()
    
        grant_url = driver.current_url
        driver.get(grant_url)
        driver.implicitly_wait(2)
        grant = driver.find_element(By.CLASS_NAME, "grant")

        grant_dict = {}

        grant_dict['grantee_name'] = safe_find_element_text(grant, By.TAG_NAME, "h1")
        grant_dict['grant_description'] = safe_find_element_text(grant, By.TAG_NAME, "h3")
        grant_dict['grantee_website'] = safe_find_element_attribute(grant, By.CLASS_NAME, "aboutgrantee-extra-value", 'href')
        grant_dict['grant_address'] = safe_find_element_text(grant, By.CLASS_NAME, "aboutgrantee-address")
        detail= grant.find_element(By.CLASS_NAME, "highlights-list")
        for label, value in zip(detail.find_elements(By.CLASS_NAME, "highlights-label"),detail.find_elements(By.CLASS_NAME, "highlights-value")):
            grant_dict[label.text] = value.text
        try:
            strategy = grant.find_element(By.CLASS_NAME, "highlights-strategies")
            strategies = []
            for value in strategy.find_elements(By.CLASS_NAME, "highlight-strategy"):
                strategies.append(value.text)
            grant_dict['strategies'] = strategies
        except Exception:
            grant['strategies'] = ""
        all_pages.append(grant_dict)

        driver.back()
        driver.implicitly_wait(2)

    time.sleep(5)

In [ ]:
len(all_pages)

In [76]:
with open("hewlettdata.json", "a") as file:
    json.dump(all_pages, file, indent=4)